In [1]:
import pandas as pd

#### DataDrift - check whether 2 distributions are similar or not

- we can use evidentlyai.com is used to check the data drift but works slow when data is large
- Alternatively we can us Tensorflow Data Validation library as it is better performance wise.
- or build custom function for checking data drift. we are using kolgonomorov-srinov test
    - Null Hpyothesis :  if 2 distributions are same distributions are identical(p>alpha=0.5)
    - Alternative Hypothesis : if 2 dristribution are not identical then there is data drift(p< alpha=0.5)



In [2]:
from scipy.stats import ks_2samp

In [5]:
import numpy as np
d1 = np.arange(10)
d2 = np.arange(10)
ks_2samp(d1,d2)

KstestResult(statistic=0.0, pvalue=1.0, statistic_location=0, statistic_sign=1)

d1 and d2 are not from the same distribution 

In [7]:
d1 = np.arange(100,200)
d2 = np.arange(10)
ks_2samp(d1,d2)

KstestResult(statistic=1.0, pvalue=4.2646072253826637e-14, statistic_location=9, statistic_sign=-1)

observe that P-value is almost 0 because d1 and d2 are not in the same range

Let's do it for our train and test data

In [25]:
train_file_path = "C:/projects/Project1/APS-sensor-fault-detection/artifact/06_17_2023_16_58_54/data_ingestion/ingested/train.csv"
test_file_path = "C:/projects/Project1/APS-sensor-fault-detection/artifact/06_17_2023_16_58_54/data_ingestion/ingested/test.csv"

In [27]:
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

In [28]:
train_df.columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [29]:
test_df.columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [30]:
d1 = train_df['aa_000']
d2 = test_df['aa_000']
ks_2samp(d1,d2)

KstestResult(statistic=0.010059309559517837, pvalue=0.7122504505702205, statistic_location=91132, statistic_sign=1)

now observe that there is 71% similarity

In [44]:
# get the columns names where drift or identical pvalue is 5%
def get_drift_report(base_df, current_df, threshold=0.05):
    report = {}
    for column in base_df.columns:
        d1 = base_df[column]
        d2 = current_df[column]
        is_same_dist = ks_2samp(d1,d2)
        if threshold<=is_same_dist.pvalue:
            is_found=False
        else:
            is_found=True
        report.update({column:{
            "p_value":float(is_same_dist.pvalue),
            "drift_status":is_found}})
    return report

In [45]:
get_drift_report(train_df, test_df)

{'class': {'p_value': 1.0, 'drift_status': False},
 'aa_000': {'p_value': 0.7122504505702205, 'drift_status': False},
 'ac_000': {'p_value': 0.8119138636579566, 'drift_status': False},
 'ad_000': {'p_value': 0.0460108064909619, 'drift_status': True},
 'ae_000': {'p_value': 0.9867153765235301, 'drift_status': False},
 'af_000': {'p_value': 0.9826254099936215, 'drift_status': False},
 'ag_000': {'p_value': 1.0, 'drift_status': False},
 'ag_001': {'p_value': 0.9999999999916019, 'drift_status': False},
 'ag_002': {'p_value': 0.9999855306804062, 'drift_status': False},
 'ag_003': {'p_value': 0.9606197153549241, 'drift_status': False},
 'ag_004': {'p_value': 0.618453213992255, 'drift_status': False},
 'ag_005': {'p_value': 0.39503244033633467, 'drift_status': False},
 'ag_006': {'p_value': 0.7307990726797906, 'drift_status': False},
 'ag_007': {'p_value': 0.3787602693004445, 'drift_status': False},
 'ag_008': {'p_value': 0.6030778159207568, 'drift_status': False},
 'ag_009': {'p_value': 0.43

Write the report to yaml file

In [46]:
from sensor.utils.main_utils import write_yaml_file

In [47]:
data_drift_report = get_drift_report(train_df, test_df)

In [48]:
import os
write_yaml_file(file_path=os.path.join(os.getcwd(),"report.yaml"), content=data_drift_report, replace=True
)

### Mapping

In [15]:
from sensor.ml.model.estimator import TargetValueMapping

ImportError: cannot import name 'TargetValueMapping' from 'sensor.ml.model.estimator' (c:\projects\project1\aps-sensor-fault-detection\sensor\ml\model\estimator.py)